##### ID2214 Assignment 1 Group no. 15
### Project members: 
[Stefano Perenzoni, perenz@kth.se]
[Stephen Moran, smoran@kth.se]
[Christian Antonio, cantonio@kth.se]

### Declaration:
By submitting this solution, it is hereby declared that all individuals listed above have contributed to the solution, either with code that appear in the final solution below, or with code that has been evaluated and compared to the final solution, but for some reason has been excluded. It is also declared that all project members fully understand all parts of the final solution and can explain it upon request.

It is furthermore declared that the code below is a contribution by the project members only, and specifically that no part of the solution has been copied from any other source (except for lecture slides at the course ID2214/FID3214) and no part of the solution has been provided by someone not listed as project member above.

It is furthermore declared that it has been understood that no other library/package than the Python 3 standard library, NumPy and pandas may be used in the solution for this assignment.

### Instructions
All parts of the assignment starting with number 1 below are mandatory. Satisfactory solutions
will give 1 point (in total). If they in addition are good (all parts work more or less 
as they should), completed on time (submitted before the deadline in Canvas) and according
to the instructions, together with satisfactory solutions of all parts of the assignment starting 
with number 2 below, then the assignment will receive 2 points (in total).

Note that you do not have to develop the code directly within the notebook
but may instead copy the comments and test cases to a more convenient development environment
and when everything works as expected, you may paste your functions into this
notebook, do a final testing (all cells should succeed) and submit the whole notebook 
(a single file) in Canvas (do not forget to fill in your group number and names above).

## Load NumPy and pandas

In [1]:
import numpy as np
import pandas as pd

## 1a. Create and apply column filter

In [55]:
# Insert the functions create_column_filter and apply_column_filter below (after the comments)
#
# Input to create_column_filter:
# df - a dataframe (where the column names "CLASS" and "ID" have special meaning)
#
# Output from create_filter:
# df            - a new dataframe, where columns, except "CLASS" and "ID", containing only missing values 
#                 or only one unique value (apart from the missing values) have been dropped
# column_filter - a list of the names of the remaining columns, including "CLASS" and "ID"
#
# Hint 1: First copy the input dataframe and modify the copy (the input dataframe should be kept unchanged)
#
# Hint 2: Iterate through all columns and consider to drop a column only if it is not labeled "CLASS" or "ID"
#
# Hint 3: You may check the number of unique (non-missing) values in a column by applying the pandas functions
#         dropna and unique to drop missing values and get the unique (remaining) values
#
def create_column_filter(df): 
    exempt = ["CLASS"]
    col_filter = exempt.copy()
    for col in df.columns: 
        if col not in exempt: 
            if (not df[col].isnull().values.all()) and (df[col].nunique() > 1): 
                col_filter.append(col)
    filtered = df.copy()
    filtered = filtered.loc[:, col_filter]
    return filtered, col_filter

# Input to apply_column_filter:
# df            - a dataframe
# column_filter - a list of the names of the columns to keep (see above)
#
# Output from apply_column_filter:
# df - a new dataframe, where each column that is not included in column_filter has been dropped
#
# Hint 1: First copy the input dataframe and modify the copy (the input dataframe should be kept unchanged)


def apply_column_filter(df, column_filter):
    return df[column_filter]


In [56]:
# Test your code (leave this part unchanged)

df = pd.DataFrame({"CLASS":[1,0,1,0,1],"A":[1,2,np.nan,4,5],"B":[1,1,1,1,np.nan],"C":["h","h",np.nan,"i","h"],"D":[np.nan,np.nan,np.nan,np.nan,np.nan]})

print(df.columns)
filtered_df, column_filter = create_column_filter(df)


new_df = pd.DataFrame({"CLASS":[1,0,0],"A":[4,5,6],"B":[1,2,1],"C":[np.nan,np.nan,np.nan],"D":[np.nan,4,5]})

filtered_new_df = apply_column_filter(new_df,column_filter)

display("df",df)
display("filtered_df",filtered_df)
display("new_df",new_df)
display("filtered_new_df",filtered_new_df)

Index(['CLASS', 'A', 'B', 'C', 'D'], dtype='object')


'df'

,CLASS,A,B,C,D
0,1,1.0,1.0,h,NaN
1,0,2.0,1.0,h,NaN
2,1,NaN,1.0,NaN,NaN
3,0,4.0,1.0,i,NaN
4,1,5.0,NaN,h,NaN


'filtered_df'

,CLASS,A,C
0,1,1.0,h
1,0,2.0,h
2,1,NaN,NaN
3,0,4.0,i
4,1,5.0,h


'new_df'

,CLASS,A,B,C,D
0,1,4,1,NaN,NaN
1,0,5,2,NaN,4.0
2,0,6,1,NaN,5.0


'filtered_new_df'

,CLASS,A,C
0,1,4,NaN
1,0,5,NaN
2,0,6,NaN


### Comment on assumptions, things that do not work properly, etc.

Using the method .fillna(axis=1, how='all', subset=['A','B','C''D']) we got an unexpected key error on ['A','B','C''D'] 

So, we decided to check singularly each column

## 1b. Create and apply normalization

In [62]:
# Insert the functions create_normalization and apply_normalization below (after the comments)
#
# Input to create_normalization:
# df: a dataframe (where the column names "CLASS" and "ID" have special meaning)
# normalizationtype: "minmax" (default) or "zscore"
#
# Output from create_normalization:
# df            - a new dataframe, where each numeric value in a column has been replaced by a normalized value
# normalization - a mapping (dictionary) from each column name to a triple, consisting of
#                ("minmax",min_value,max_value) or ("zscore",mean,std)
#
# Hint 1: First copy the input dataframe and modify the copy (the input dataframe should be kept unchanged)
#
# Hint 2: Consider columns of type "float" or "int" only (and which are not labeled "CLASS" or "ID"),
#         the other columns should remain unchanged
#
# Hint 3: Take a close look at the lecture slides on data preparation
def create_normalization(df, normalizationtype='minmax'):
    to_rtn = df.copy()
    norm_dict = {}
    colu = to_rtn.columns
    print(colu)
    for col in colu:
        #Check if col is numeric
        if pd.api.types.is_numeric_dtype(to_rtn[col]) and col not in ['CLASS', 'ID']:
            if normalizationtype == 'minmax':
                max = to_rtn[col].max()
                min = to_rtn[col].min()
                norm_dict[col] = (normalizationtype, min, max)
                #Normalize using minamx
                to_rtn[col] = to_rtn[col].apply(lambda x: (x-min)/(max-min))
            if normalizationtype == 'zscore':
                mean = to_rtn[col].mean()
                std = to_rtn[col].std()
                norm_dict[col] = ('zscore', mean, std)
                #Normalizing using zscore
                to_rtn[col] = to_rtn[col].apply(lambda x: (x-mean)/std)
    return to_rtn, norm_dict


#
# Input to apply_normalization:
# df            - a dataframe
# normalization - a mapping (dictionary) from column names to triples (see above)
#
# Output from apply_normalization:
# df - a new dataframe, where each numerical value has been normalized according to the mapping
#
# Hint 1: First copy the input dataframe and modify the copy (the input dataframe should be kept unchanged)
#
# Hint 2: For minmax-normalization, you may consider to limit the output range to [0,1]

def apply_normalization(df, norm_dict):
  to_rtn = df.copy()
  for col in to_rtn.columns:
    if pd.api.types.is_numeric_dtype(to_rtn[col]) and col not in ['CLASS', 'ID']:
        min = norm_dict[col][1]
        mean = norm_dict[col][1]
        max = std = norm_dict[col][2]
        normtype = norm_dict[col][0]
        if normtype == 'minmax':
            val_list = []
            # Calculate the normalized value in the range [0,1]
            for x in to_rtn[col]:
                v = (x-min)/(max-min)
                if v > 1.0:
                    v = 1
                if v < 0.0:
                    v = 0
                val_list.append(v)
            to_rtn[col] = val_list
            #to_rtn[col] = to_rtn[col].apply(lambda x: (x-min)/(max-min)) #Have to consider the [0,1] range
        if normtype == 'zscore':
            #Normalize with zscore
            to_rtn[col] = to_rtn[col].apply(lambda x: (x-mean)/std)
  return to_rtn



In [63]:
# Test your code (leave this part unchanged)

glass_train_df = pd.read_csv("glass_train.csv")

glass_test_df = pd.read_csv("glass_test.csv")

#print(glass_train_df['RI'])

glass_train_norm, normalization = create_normalization(glass_train_df,normalizationtype="zscore")
print("normalization:\n")
for f in normalization:
    print("{}:{}".format(f,normalization[f]))

print()
    
glass_test_norm = apply_normalization(glass_test_df,normalization)
display("glass_test_norm",glass_test_norm)

Index(['ID', 'RI', 'Na', 'Mg', 'Al', 'Si', 'K', 'Ca', 'Ba', 'Fe', 'CLASS'], dtype='object')
normalization:

RI:('zscore', 1.518123551401869, 0.002867216865111231)
Na:('zscore', 13.45981308411215, 0.8892682823200198)
Mg:('zscore', 2.4488785046728974, 1.543675060576804)
Al:('zscore', 1.5069158878504674, 0.5090453722131358)
Si:('zscore', 72.66504672897196, 0.793964703639898)
K:('zscore', 0.5295327102803739, 0.8716200305715377)
Ca:('zscore', 8.997102803738317, 1.3330076712441825)
Ba:('zscore', 0.20794392523364486, 0.5266586347727279)
Fe:('zscore', 0.05869158878504673, 0.10187935995670466)



'glass_test_norm'

,ID,RI,Na,Mg,Al,Si,K,Ca,Ba,Fe,CLASS
0,101,-0.548808,-0.798199,0.259848,-0.131454,0.761940,0.046428,-0.155365,-0.185972,1.583328,2
1,104,3.183034,0.382547,0.454190,-1.663734,-2.638715,-0.515744,1.982657,-0.394836,-0.576089,2
2,44,1.386867,0.303831,0.901175,-1.545866,-1.139908,-0.412488,0.557309,-0.394836,-0.576089,1
3,17,-0.098894,-0.876915,0.791048,-0.681503,0.560419,0.092319,-0.222882,-0.394836,-0.576089,1
4,81,-0.768533,-0.674502,0.693878,1.204380,-0.006356,0.184102,-0.770515,-0.394836,-0.576089,2
...,...,...,...,...,...,...,...,...,...,...,...
102,80,-0.775509,-0.719483,0.693878,0.772199,0.245544,0.184102,-0.770515,-0.394836,-0.576089,2
103,141,-0.426738,-0.145977,0.706834,0.202505,-0.157497,0.172629,-0.665490,-0.394836,-0.576089,2
104,14,-0.224452,-0.674502,0.719790,-0.465412,0.686370,0.012009,-0.462940,-0.394836,1.092551,1
105,132,2.795899,0.270095,-1.586395,-0.288611,-1.794849,-0.389542,3.332987,-0.394836,0.405464,2


### Comment on assumptions, things that do not work properly, etc.

Works as expected

## 1c. Create and apply imputation

In [65]:
# Insert the functions create_imputation and apply_imputation below (after the comments)
#
# Input to create_imputation:
# df: a dataframe (where the column names "CLASS" and "ID" have special meaning)
#
# Output from create_imputation:
# df         - a new dataframe, where each missing numeric value in a column has been replaced by the mean of that column 
#              and each missing categoric value in a column has been replaced by the mode of that column
# imputation - a mapping (dictionary) from column name to value that has replaced missing values
#
# Hint 1: First copy the input dataframe and modify the copy (the input dataframe should be kept unchanged)
#
# Hint 2: Handle columns of type "float" or "int" only (and which are not labeled "CLASS" or "ID") in one way
#         and columns of type "object" and "category" in other ways
#
# Hint 3: Consider using the pandas functions mean and mode respectively, as well as fillna
#
# Hint 4: In the rare case of all values in a column being missing*, replace numeric values with 0,
#         object values with "" and category values with the first category (cat.categories[0])  
#
#         *Note that this will not occur if the previous column filter function has been applied
def create_imputation(df):
    to_rtn = df.copy()
    imp_dict = {}
    for col in to_rtn.columns:
        if col not in ['CLASS', 'ID']:
            if to_rtn[col].dtypes == "int" or to_rtn[col].dtype == "float":
                val = to_rtn[col].mean()
                to_rtn[col] = to_rtn[col].fillna(value=val)
            else:
                #Get first value of the mode, alternatively np.random could be used
                val = to_rtn[col].mode()[0]
                to_rtn[col] = to_rtn[col].fillna(value=val)
            imp_dict[col] = val
    #to_rtn = to_rtn.fillna(value=imp_dict)
    return to_rtn, imp_dict



#
# Input to apply_imputation:
# df         - a dataframe
# imputation - a mapping (dictionary) from column name to value that should replace missing values
#
# Output from apply_imputation:
# df - a new dataframe, where each missing value has been replaced according to the mapping
#
# Hint 1: First copy the input dataframe and modify the copy (the input dataframe should be kept unchanged)
#
# Hint 2: Consider using fillna
def apply_imputation(df, imputation):
    to_rtn = df.copy()
    #No check for columns type because only rights type are included in the imputation dictionary
    to_rtn = to_rtn.fillna(value=imputation)
    return to_rtn


In [66]:
# Test your code (leave this part unchanged)

anneal_train_df = pd.read_csv("anneal_train.csv")
anneal_test_df = pd.read_csv("anneal_test.csv")

anneal_train_imp, imputation = create_imputation(anneal_train_df)
anneal_test_imp = apply_imputation(anneal_test_df,imputation)

print("Imputation:\n")
for f in imputation:
    print("{}:{}".format(f,imputation[f]))

print("\nNo. of replaced missing values in training data:\n{}".format(anneal_train_imp.count()-anneal_train_df.count()))
print("\nNo. of replaced missing values in test data:\n{}".format(anneal_test_imp.count()-anneal_test_df.count()))

Imputation:

family:TN
product-type:C
steel:A
carbon:3.859688195991091
hardness:13.084632516703786
temper_rolling:T
condition:S
formability:2.2517482517482517
strength:26.302895322939868
non-ageing:N
surface-finish:P
surface-quality:E
enamelability:1.7142857142857142
bc:Y
bf:Y
bt:Y
bw/me:B
bl:Y
m:nan
chrom:C
phos:P
cbond:Y
marvi:nan
exptl:nan
ferro:Y
corr:nan
blue-bright-varn-clean:B
lustre:Y
jurofm:nan
s:nan
p:nan
shape:SHEET
thick:1.1911937639198218
width:769.4917594654788
len:1229.293986636971
oil:Y
bore:35.18930957683742
packing:3.0

No. of replaced missing values in training data:
family                    382
product-type                0
steel                      43
carbon                      0
hardness                    0
temper_rolling            374
condition                 160
formability               163
strength                    0
non-ageing                391
surface-finish            444
surface-quality           128
enamelability             442
bc               

### Comment on assumptions, things that do not work properly, etc.

Works as expected

## 1d. Create and apply discretization

In [75]:
# Insert the functions create_bins and apply_bins below
#
# Input to create_bins:
# df      - a dataframe
# nobins  - no. of bins (default = 10)
# bintype - either "equal-width" (default) or "equal-size" 
#
# Output from create_bins:
# df      - a new dataframe, where each numeric feature value has been replaced by a categoric (corresponding to some bin)
# binning - a mapping (dictionary) from column name to bins (threshold values for the bin)
#
# Hint 1: First copy the input dataframe and modify the copy (the input dataframe should be kept unchanged)
#
# Hint 2: Discretize columns of type "float" or "int" only (and which are not labeled "CLASS" or "ID")
#
# Hint 3: Consider using pd.cut and pd.qcut respectively, with labels=False and retbins=True
#
# Hint 4: Set all columns in the new dataframe to be of type "category"
#
# Hint 5: Set the categories of the discretized features to be [0,...,nobins-1]
#
# Hint 6: Change the first and the last element of each binning to -np.inf and np.inf respectively 
#
def create_bins(df, nobins=10, bintype='equal-width'):
    to_rtn = df.copy()
    bins = {}
    for col in to_rtn.columns:
        if col != "CLASS" and col != "ID" and df[col].dtype in ["float64", "float32", "int64", "int32"]:
            if bintype == "equal-width":
                to_rtn[col], binR = pd.cut(to_rtn[col],nobins,retbins=True,duplicates="drop",labels=False)
                bins[col] = binR    
            elif bintype == "equal-size":
                to_rtn[col], binR = pd.qcut(to_rtn[col],q=nobins,retbins=True,duplicates="drop",labels=False)
                bins[col] = binR
            to_rtn[col] = to_rtn[col].astype("category")
            to_rtn[col] = to_rtn[col].cat.set_categories([str(i) for i in to_rtn[col].cat.categories], rename = True)
            #Infinity edges
            bins[col][0] = -np.inf
            bins[col][-1] = np.inf
        else:
            to_rtn[col] = to_rtn[col].astype('category')
    return to_rtn, bins

         

# Input to apply_bins:
# df      - a dataframe
# binning - a mapping (dictionary) from column name to bins (threshold values for the bin)
#
# Output from apply_bins:
# df - a new dataframe, where each numeric feature value has been replaced by a categoric (corresponding to some bin)
#
# Hint 1: First copy the input dataframe and modify the copy (the input dataframe should be kept unchanged)
#
# Hint 2: Consider using pd.cut 
#
# Hint 3: Set all columns in the new dataframe to be of type "category"
#
# Hint 4: Set the categories of the discretized features to be [0,...,nobins-1]
def apply_bins(df, binning): 
    exempt = ["ID", "CLASS"]
    binned_df = df.copy()
    for col in binned_df.columns: 
        if col not in exempt: 
            if binned_df[col].dtype == "int64" or binned_df[col].dtype == "float64": 
                bins = binning[col]
                bin_length = len(bins) - 1
                labels = np.arange(0, bin_length, 1)
                binned_df[col] = pd.cut(binned_df[col], bins, labels=labels)
                binned_df[col] = binned_df[col].astype('category')
    return binned_df


In [76]:
# Test your code  (leave this part unchanged)

glass_train_df = pd.read_csv("glass_train.csv")

glass_test_df = pd.read_csv("glass_test.csv")

glass_train_disc, binning = create_bins(glass_train_df,nobins=10,bintype="equal-width")
print("binning:")
for f in binning:
    print("{}:{}".format(f,binning[f]))

print()    
glass_test_disc = apply_bins(glass_test_df,binning)
display("glass_test_disc",glass_test_disc)

binning:
RI:[    -inf 1.513304 1.515298 1.517292 1.519286 1.52128  1.523274 1.525268
 1.527262 1.529256      inf]
Na:[  -inf 11.236 11.742 12.248 12.754 13.26  13.766 14.272 14.778 15.284
    inf]
Mg:[ -inf 0.449 0.898 1.347 1.796 2.245 2.694 3.143 3.592 4.041   inf]
Al:[ -inf 0.565 0.84  1.115 1.39  1.665 1.94  2.215 2.49  2.765   inf]
Si:[  -inf 70.347 70.884 71.421 71.958 72.495 73.032 73.569 74.106 74.643
    inf]
K:[ -inf 0.621 1.242 1.863 2.484 3.105 3.726 4.347 4.968 5.589   inf]
Ca:[  -inf  6.355  7.28   8.205  9.13  10.055 10.98  11.905 12.83  13.755
    inf]
Ba:[ -inf 0.315 0.63  0.945 1.26  1.575 1.89  2.205 2.52  2.835   inf]
Fe:[ -inf 0.037 0.074 0.111 0.148 0.185 0.222 0.259 0.296 0.333   inf]



'glass_test_disc'

,ID,RI,Na,Mg,Al,Si,K,Ca,Ba,Fe,CLASS
0,101,2,3,6,4,6,0,3,0,5,2
1,104,7,6,7,1,1,0,6,0,0,2
2,44,5,5,8,1,3,0,4,0,0,1
3,17,3,3,8,3,6,0,3,0,0,1
4,81,2,4,7,6,5,1,2,0,0,2
...,...,...,...,...,...,...,...,...,...,...,...
102,80,2,4,7,5,5,1,2,0,0,2
103,141,2,5,7,4,5,1,2,0,0,2
104,14,3,4,7,3,6,0,3,0,4,1
105,132,7,5,0,3,2,0,8,0,2,2


### Comment on assumptions, things that do not work properly, etc.

Works as expected

## 1e. Create and apply one-hot encoding

In [79]:
# Insert the functions create_one_hot and apply_one_hot below
#
# Input to create_one_hot:
# df: a dataframe
#
# Output from create_one_hot:
# df      - a new dataframe, where each categoric feature has been replaced by a set of binary features 
#           (as many new features as there are possible values)
# one_hot - a mapping (dictionary) from column name to a set of categories (possible values for the feature)
def create_one_hot(df):
    to_rtn = df
    enc = {}
    for col in [el for el in to_rtn.columns if el not in ['CLASS', 'ID']]:
        #Check for columns type
        if str(to_rtn.dtypes[col]) == "category" or str(to_rtn.dtypes[col]) == "object":
            #Convert columns to category
            to_rtn[col] = to_rtn[col].astype("category")
            #For each column, get the list of categories
            enc[col] = list(to_rtn[col].cat.categories)
            for i in enc[col]:
                tit = col + '_' + str(i) #Name of the new column
                col_enc = to_rtn[col] == i
                col_enc = col_enc.astype("int")
                df_enc = col_enc
            df_enc = df_enc.drop(axis=1, columns=col)
            
    return df_enc, enc
            
#
# Hint 1: First copy the input dataframe and modify the copy (the input dataframe should be kept unchanged)
#
# Hint 2: Consider columns of type "object" or "category" only (and which are not labeled "CLASS" or "ID")
#
# Hint 3: Consider creating new column names by merging the original column name and the categorical value
#
# Hint 4: Set all new columns to be of type "float"
#
# Hint 5: Do not forget to remove the original categoric feature
#
# Input to apply_one_hot:
# df      - a dataframe
# one_hot - a mapping (dictionary) from column name to categories
#
# Output from apply_one_hot:
# df - a new dataframe, where each categoric feature has been replaced by a set of binary features
#
# Hint: See the above Hints
def apply_one_hot(df,one_hot):
    df_new = df.copy()
    for col in df.columns:
        if col in one_hot:    
            for i in one_hot[col]:
                name = col + "_" + str(i)
                new_col = df[col]==i
                new_col = pd.Series(new_col.astype("int"))
                df_new[name] = new_col
            df_new = df_new.drop(columns = col, axis = 1)
            
    return df_new


In [80]:
# Test your code  (leave this part unchanged)

train_df = pd.read_csv("tic-tac-toe_train.csv")

new_train, one_hot = create_one_hot(train_df)

test_df = pd.read_csv("tic-tac-toe_test.csv")

new_test_df = apply_one_hot(test_df,one_hot)
display("new_test_df",new_test_df)

'new_test_df'

,CLASS,top-left-square_b,top-left-square_o,top-left-square_x,top-middle-square_b,top-middle-square_o,top-middle-square_x,top-right-square_b,top-right-square_o,top-right-square_x,...,middle-right-square_x,bottom-left-square_b,bottom-left-square_o,bottom-left-square_x,bottom-middle-square_b,bottom-middle-square_o,bottom-middle-square_x,bottom-right-square_b,bottom-right-square_o,bottom-right-square_x
0,positive,0,1,0,0,1,0,0,0,1,...,1,0,1,0,0,0,1,0,0,1
1,positive,0,1,0,0,1,0,0,0,1,...,1,0,0,1,0,1,0,0,0,1
2,positive,1,0,0,0,0,1,0,0,1,...,0,0,0,1,1,0,0,0,1,0
3,negative,0,1,0,0,0,1,0,0,1,...,0,0,0,1,0,0,1,0,1,0
4,positive,0,0,1,0,1,0,0,1,0,...,0,1,0,0,0,1,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
474,positive,1,0,0,1,0,0,0,0,1,...,1,0,1,0,1,0,0,0,0,1
475,negative,0,1,0,0,1,0,0,1,0,...,1,1,0,0,0,0,1,1,0,0
476,positive,0,0,1,1,0,0,0,0,1,...,0,0,0,1,0,1,0,0,1,0
477,positive,0,0,1,1,0,0,0,0,1,...,0,0,0,1,0,1,0,0,1,0


### Comment on assumptions, things that do not work properly, etc.

Works as expected

## 1f. Divide a dataset into a training and a test set

In [84]:
# Insert the function split below
#
# Input to split:
# df           - a dataframe
# testfraction - a float in the range (0,1) (default = 0.5)
#
# Output from split:
# trainingdf - a dataframe consisting of a random sample of (1-testfraction) of the rows in df
# testdf     - a dataframe consisting of the rows in df that are not included in trainingdf
#
# Hint: You may use np.random.permutation(df.index) to get a permuted list of indexes where a 
#       prefix corresponds to the test instances, and the suffix to the training instances 
def split(df, testfraction = 0.5): 
    df_length = len(df)
    cut_off = df_length * testfraction
    cut_off = int(cut_off)
    indexes = np.random.permutation(df.index)
    test_indexes = indexes[0:cut_off]
    training_indexes = indexes[cut_off:]
    trainingdf = df.iloc[training_indexes, :]
    testdf = df.iloc[test_indexes, :]
    return trainingdf, testdf


In [87]:
# Test your code  (leave this part unchanged)

glass_df = pd.read_csv("glass.csv")

glass_train, glass_test = split(glass_df,testfraction=0.25)

print("Training IDs:\n{}".format(glass_train["ID"].values))

print("\nTest IDs:\n{}".format(glass_test["ID"].values))

print("\nOverlap: {}".format(set(glass_train["ID"]).intersection(set(glass_test["ID"]))))

Training IDs:
[150 128   6 117 124 210  27  22 197 104  71 108  26  19 189 106 135  16
  96  83 161  53  84 211 133 125  76  43  82 116   9 107 185 195 175  29
 129 183  47  18  34 209   4  28  36  25  42 169 193 163  49 109 199  95
  32 182  61 132 188 179  79 160 186 126 120  59  13  41 143  46 102 196
 152  90   3 172 194 100  86 177  52  80  63  48 113  64  58 192  56   7
 162  37 156  98  68  72  10 121 139  15 144 134  51 167 155  21  55 137
  11 141  14 164 190  44  23  20  33 203 157 202 101  60 213 118 184 130
 112 166 145 110 174   1 140  17  24 115  69 131 187 147 159  74 151  57
   8 149  78  66 205  12 176  75 105 148 103 201  77 200 178 170 122]

Test IDs:
[191  65 146  81   5 127 208  67  73  97 165 138 214  54  70 136 173 153
  30 207 123   2 154 119 158  87  92  50  40  93  91 198  88  62 142  94
  35  38 212 171  31  89 180  99  39 204 114  85 206 111 181  45 168]

Overlap: set()


### Comment on assumptions, things that do not work properly, etc.

Works as expected
If the calculated index is a float number we round down to the nearest integer

## 1g. Calculate accuracy of a set of predictions

In [88]:
# Insert the function accuracy below
#
# Input to accuracy:
# df            - a dataframe with class labels as column names and each row corresponding to
#                 a prediction with estimated probabilities for each class
# correctlabels - an array (or list) of the correct class label for each prediction
#                 (the number of correct labels must equal the number of rows in df)
#
# Output from accuracy:
# accuracy - the fraction of cases for which the predicted class label coincides with the correct label
#
# Hint: In case the label receiving the highest probability is not unique, you may
#       resolve that by picking the first (as ordered by the column names) or 
#       by randomly selecting one of the labels with highest probaility.
def accuracy(df, correctlabels):
    labels = df.idxmax(axis=1)
    truelabels = (labels == correctlabels).sum()
    accuracy = truelabels/len(df)
    
    return accuracy


In [89]:
# Test your code  (leave this part unchanged)

predictions = pd.DataFrame({"A":[0.5,0.5,0.5,0.25,0.25],"B":[0.5,0.25,0.25,0.5,0.25],"C":[0.0,0.25,0.25,0.25,0.5]})
display("predictions",predictions)

'predictions'

,A,B,C
0,0.50,0.50,0.00
1,0.50,0.25,0.25
2,0.50,0.25,0.25
3,0.25,0.50,0.25
4,0.25,0.25,0.50


In [90]:
correctlabels = ["B","A","B","B","C"]

print("Accuracy: {}".format(accuracy(predictions,correctlabels))) # Note that depending on how ties are resolved the accuracy may be 0.6 or 0.8

Accuracy: 0.6


### Comment on assumptions, things that do not work properly, etc.

Works as expected

## 2a. Divide a dataset into a number of folds

In [91]:
# Insert the function folds below
#
# Input to folds:
# df      - a dataframe
# nofolds - an integer greater than 1 (default = 10)
#
# Output from folds:
# folds - a list (of length = nofolds) dataframes consisting of random non-overlapping, 
#         approximately equal-sized subsets of the rows in df
#
# Hint: You may use np.random.permutation(df.index) to get a permuted list of indexes from which a 
#       prefix corresponds to the test instances, and the suffix to the training instances 
def folds(df, nofolds=10):
    to_rtn = df.copy()
    to_rtn = np.random.permutation(to_rtn.index)
    fold = []
    
    for i in range(nofolds):
            fold.append(df[int(i*len(df)/nofolds):int((i+1)*len(df)/nofolds)])
            
    return fold


In [92]:
# Test your code  (leave this part unchanged)

glass_df = pd.read_csv("glass.csv")

glass_folds = folds(glass_df,nofolds=10)

fold_sizes = [len(f) for f in glass_folds]

print("Fold sizes:{}\nTotal no. instances: {}".format(fold_sizes,sum(fold_sizes)))

Fold sizes:[21, 21, 22, 21, 22, 21, 21, 22, 21, 22]
Total no. instances: 214


### Comment on assumptions, things that do not work properly, etc.

Works as expected

## 2b. Calculate Brier score of a set of predictions

In [93]:
# Insert the function brier_score below
#
# Input to brier_score:
# df            - a dataframe with class labels as column names and each row corresponding to
#                 a prediction with estimated probabilities for each class
# correctlabels - an array (or list) of the correct class label for each prediction
#                 (the number of correct labels must equal the number of rows in df)
#
# Output from brier_score:
# brier_score - the average square error of the predicted probabilties 
#
# Hint: Compare each predicted vector to a vector for each correct label, which is all zeros except 
#       for at the index of the correct class. The index can be found using np.where(df.columns==l)[0] 
#       where l is the correct label.
def brier_score(df, corretlabels): 
    label_df = pd.get_dummies(correctlabels)
    brier_score = np.mean(np.sum((df - label_df)**2, axis=1))
    
    return brier_score


In [94]:
# Test your code  (leave this part unchanged)

predictions = pd.DataFrame({"A":[0.5,0.5,0.5,0.25,0.25],"B":[0.5,0.25,0.25,0.5,0.25],"C":[0.0,0.25,0.25,0.25,0.5]})

correctlabels = ["B","A","B","B","C"]

print("Brier score: {}".format(brier_score(predictions,correctlabels)))

Brier score: 0.5


### Comment on assumptions, things that do not work properly, etc.

Works as expected

## 2c. Calculate AUC of a set of predictions

In [95]:
# Insert the function auc below
#
# Input to auc:
# df            - a dataframe with class labels as column names and each row corresponding to
#                 a prediction with estimated probabilities for each class
# correctlabels - an array (or list) of the correct class label for each prediction
#                 (the number of correct labels must equal the number of rows in df)
#
# Output from auc:
# auc - the weighted area under ROC curve
#
# Hint 1: Calculate the binary AUC first for each class label c, i.e., treating the
#         predicted probability of this class for each instance as a score; the true positives
#         are the ones belonging to class c and the false positives the rest
#
# Hint 2: When calculating the binary AUC, first find the scores of the true positives and then
#         the scores of the true negatives
#
# Hint 3: You may use a dictionary with a mapping from each score to an array of two numbers; 
#         the number of true positives with this score and the number of true negatives with this score
#
# Hint 4: Created a (reversely) sorted (on the scores) list of pairs from the dictionary and
#         iterate over this to additively calculate the AUC
#
# Hint 5: For each pair in the above list, there are three cases to consider; the no. of false positives
#         is zero, the no. of true positives is zero, and both are non-zero
#
# Hint 6: Calculate the weighted AUC by summing the individual AUCs weighted by the relative
#         frequency of each class (as estimated from the correct labels)
def auc(df, correctlabels):   
    cols = df.columns
    cor_list = [(c == cols) for c in correctlabels]
    #print(cor_list)
    correct_filter = pd.DataFrame(cor_list, columns=cols)

    # Calculate binary AUC for each class label
    # Treating the predicted probability of this class for each instance as a score
    AUC = 0
    for col in df.columns:     
        # Map from each score to an array with number of true positives and true negative
        class_score = {score: [0, 0] for score in df[col]} # [positive, negative]
        
        for i in range(len(df[col])):  
            # We find score of the true positives and then the ones of true negatives
            score = df[col][i] # probability of class col row n
            is_positive = correct_filter[col][i] == True
            class_score[score] = [class_score[score][0] + is_positive,
                                  class_score[score][1] + ~is_positive]
        #We create a single reversed list of pairs
        sort_score = sorted(class_score, reverse=True)
        sorted_list = np.array([class_score[score] for score in sort_score])
        
        class_auc = cov_tp = 0
        tp, fp = sorted_list[:, 0], sorted_list[:, 1] 
        tot_tp, tot_fp = sum(tp), sum(fp)
        
        #Evaluate the AUC considering the 3 different cases
        for i in range(len(tp)):   
            if fp[i] == 0:
                #Increase up the. y-axis
                cov_tp += tp[i]
            elif tp[i] == 0:
                #Have rectangle to calculate
                class_auc += (cov_tp/tot_tp)*(fp[i]/tot_fp)
            else: 
                class_auc += (cov_tp/tot_tp)*(fp[i]/tot_fp)+(tp[i]/tot_tp)*(fp[i]/tot_fp)/2
                cov_tp += tp[i]
                
        frequency = dict(pd.Series(correctlabels).value_counts(normalize=True))
        AUC += frequency[col]*class_auc
    return AUC

In [96]:
# Test your code  (leave this part unchanged)

predictions = pd.DataFrame({"A":[0.9,0.9,0.6,0.55],"B":[0.1,0.1,0.4,0.45]})

correctlabels = ["A","B","B","A"]

print("AUC: {}".format(auc(predictions,correctlabels)))

AUC: 0.375


In [97]:
predictions = pd.DataFrame({"A":[0.5,0.5,0.5,0.25,0.25],"B":[0.5,0.25,0.25,0.5,0.25],"C":[0.0,0.25,0.25,0.25,0.5]})

correctlabels = ["B","A","B","B","C"]

print("AUC: {}".format(auc(predictions,correctlabels)))

AUC: 0.8499999999999999


### Comment on assumptions, things that do not work properly, etc.